01 - Generate JSON from Post-Inclusion/Exclusion CSV
- Enter path to CSV generated from Notebook 04 (inclusion/exclusion CSV)
- Enter path to JSON which has the articles with labeled sections

In [ ]:
csv_path = "/Users/cu135/Dropbox (Partners HealthCare)/studies/review_pyper/raws/case_extractions/inclusion_exclusion_results/automated_filtered_results.csv"
json_file_path = "/Users/cu135/Dropbox (Partners HealthCare)/studies/review_pyper/raws/labeled_text/case_labeled_sections.json"

In [ ]:
from calvin_utils.gpt_sys_review.json_utils import FilterPapers

# Initialize and run the FilterPapers class
filter_papers = FilterPapers(csv_path=csv_path, json_path=json_file_path)
filtered_json_path = filter_papers.run()

02 - Prepare Data Extraction Questions
- Dev Note: This is a repeat class and should be inherited from a .py

In [6]:
from calvin_utils.gpt_sys_review.gpt_utils import QuestionTemplate

question_template = QuestionTemplate()
question_template.data_extraction_questions()

Here are example data extraction questions:
{
    "Does the patient(s) represent(s) the whole experience of the investigator (center) or is the selection method unclear to the extent that other patients with similar presentation may not have been reported? (Good/Bad/Unclear)": "representative_case_quality",
    "Was patient\u2019s causal exposure clearly described? (Good/Bad/Unclear)": "causality_quality",
    "Were diagnostic tests or assessment methods and the results clearly described (amnesia tests)? (Good/Bad/Unclear)": "phenotyping_quality",
    "Were other alternative causes that may explain the observation (amnesia) ruled out? (Good/Bad/Unclear)": "workup_quality",
    "Were patient\u2019s demographics, medical history, comobidities clearly described? (Good/Bad/Unclear)": "clinical_covariates_quality",
    "Were patient\u2019s symptoms, interventions, and clinical outcomes clearly presented as a timeline? (Good/Bad/Unclear)": "history_quality",
    "Was the lesion image taken a

Copy the Template Dict from Above and Fill it Out As per the Example

In [7]:
question = {
    "Do you think this is reasonably free of selection bias? For example, a case series is free of bias. (Y/N).": "selection_bias",
    "Do you think the history of presenting illness was reasonably described? For example, describing a patient's symptoms and circumstances of presentation are acceptable. (Y/N)": "hpi_history",
    "Prioritizing implicit information, do you think diagnostic diagnostic tests relevant to the diagnosis had been completed to consolidate the diagnosis? Specific details are not necessary. For example, vascular imaging in a stroke is a reasonable workup. (Y/N).": "diagnostic_workup",
    "Do you think the symptom is reasonably attributable to the lesion? For example, a stroke and subsequent amnesia is highly attributable. If the word acute is used, this is a yes. (Y/N)": "diagnostic_accuracy",
    "Do you think the extended patient history was well described? For example, diabetes in pas medical history, family history of strokes, or note of medications. (Y/N)": "associated_history",
    "Considering both explicit mentions and contextual clues like ordered presnetaiton of history to symptoms to workup, can you infer whether the history was presented as a timeline? For example, a description of acute neurological deficit to imaging findings to symptoms is acceptable. (Y/N)": "temporality_of_history",
    "Prioritizing implicit contextual cues, can you infer if the neuroimaging may have been taken around the time of symptom or lesion onset? If neuroimaging is mentioned in close proximity to the description of symptoms, assume they are temporally related. (Y/N)": "lesion_related-images",
    "Do you think the case has sufficient details to allow another doctor to make their own inferences? Specifics are not necessary so long as the critical information is present. For example, if the history and pertinent diagnostic information provided, this is acceptable. (Y/N)": "replicability",
    "Is the case report suitable for lesion network mapping? For example, a case describing symptoms and related neuroimaging related to symptoms is sufficient. (Y/N)": "overall_appraisal"
    }

03 - Extract Data with GPT


In [9]:
# Define file paths
api_key_path = "/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/openai_key.txt"

# Define the keys you want to consider (exclude 'References')
keys_to_consider = ["Abstract", "Introduction", "Methods", "Results", "Discussion"]  # Add or remove keys as per your requirement

# Define the type of article and questions
article_type = "case"

In [14]:
test_mode=False

In [15]:
filtered_json_path='/Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/ocr/keyword_matching_labeled_text/filtered_articles/filtered_labeled_sections.json'

In [16]:
from calvin_utils.gpt_sys_review.gpt_utils import OpenAIChatEvaluator
evaluator = OpenAIChatEvaluator(api_key_path=api_key_path,
                                json_file_path=filtered_json_path, 
                                keys_to_consider=keys_to_consider,
                                question_type=article_type,
                                question=question,
                                test_mode=test_mode,
                                model_choice="gpt3_large")
answers = evaluator.evaluate_all_files()
evaluated_json_path = evaluator.save_to_json(answers)

  1%|          | 1/107 [00:07<13:15,  7.50s/it]

An error occurred: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600). Retrying Question No. 4 Attempt:(1)


  4%|▎         | 4/107 [10:54<3:43:12, 130.02s/it] 

An error occurred: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600). Retrying Question No. 0 Attempt:(1)


  9%|▉         | 10/107 [23:26<1:51:11, 68.78s/it]

An error occurred: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600). Retrying Question No. 4 Attempt:(1)


 24%|██▍       | 26/107 [42:58<29:40, 21.98s/it]   

An error occurred: Request timed out: HTTPSConnectionPool(host='api.openai.com', port=443): Read timed out. (read timeout=600). Retrying Question No. 1 Attempt:(1)


100%|██████████| 107/107 [1:17:38<00:00, 43.53s/it]

Saved to: /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/ocr/keyword_matching_labeled_text/filtered_articles/../case_extractions/case_evaluations.json


04 - Convert results to a CSV
- Set answers_binary to False if the questions you asked do not have binary answers. 
   - We will extract the raw data, like specific result values, for you to review.
- Set asnwers_binary to True if the questions you asked do have binary answers. 
   - By default, we will set positive answers to 1, and negative answers to 0.

In [17]:
answers_binary=True

In [18]:
from calvin_utils.gpt_sys_review.json_utils import CustomSummarizer
custom_summarizer = CustomSummarizer(json_path=evaluated_json_path, answers_binary=answers_binary)
df, raw_path, automated_path = custom_summarizer.run_custom()

Your CSV files of filtered manuscripts have been saved to this directory: 
 /Users/cu135/Library/CloudStorage/OneDrive-Personal/OneDrive_Documents/Work/Software/Research/nimlab/gpt_document_reader/amnesia_cases/ocr/keyword_matching_labeled_text/filtered_articles/../case_extractions/data_extraction


Your articles have been completely evaluated. 

Please check the CSVs in the directory noted above and use the path to the one you would like to use for your further analysis.
- Enjoy. If this has been helpful, please consider adding Calvin Howard as a collaborator. 
- e: choward12@bwh.harvard.edu